In [1]:
import requests
import json

In [2]:
drug_name = 'Amoxicillin' #Fetch drug name from prescription (EXAMPLE LEVOSIZ 5MG SHOULD BE PUT AS LEVOSIZ-5 IN QUERY)
drug_fda = 'https://api.fda.gov/drug/drugsfda.json?search={}&limit=1'
drug_label = 'https://api.fda.gov/drug/label.json?search={}&limit=1'
drug_ndc = 'https://api.fda.gov/drug/ndc.json?search={}&limit=1' 
drug_enforcement = 'https://api.fda.gov/drug/enforcement.json?search={}&limit=1'

In [3]:
def clean_text(text):
  bad_chars = [';', ':', '!', "*",'[','"','\n',']']
  for i in bad_chars:
    text = text.replace(i, '')
  return text

In [4]:
#finding drug unique number
def unique_number(drug_name):
  url = "https://api.fda.gov/drug/drugsfda.json?search={}&limit=1".format(drug_name.upper())
  response = requests.get(url)
  unique_number = response.json()['results'][0]['openfda']['unii'][0]
  return unique_number

In [5]:
def general_information(unique_number,drug_fda):
  query = drug_fda.format(unique_number)
  response = requests.get(query)
  generic_name = json.dumps(response.json()['results'][0]['openfda']['generic_name'])
  route = json.dumps(response.json()['results'][0]['openfda']['route'])
  substance_name = json.dumps(response.json()['results'][0]['openfda']['substance_name'])
  dosage_form = json.dumps(response.json()['results'][0]['products'][0]['dosage_form'])
  json_object = json.dumps(response.json()['results'][0]['products'][0]['active_ingredients'])
  for name in range(len(json.loads(json_object))):
    active_ingredients = json.loads(json_object)[name]['name']
  
  return generic_name, route, substance_name, dosage_form, active_ingredients


In [6]:
def specific_information(unique_number,drug_label):
  query = drug_label.format(unique_number)
  response = requests.get(query)
  adverse_reactions = json.dumps(response.json()['results'][0]['adverse_reactions'])
  contraindications = json.dumps(response.json()['results'][0]['contraindications'])
  information_for_patients = json.dumps(response.json()['results'][0]['information_for_patients'])
  warnings_and_cautions = json.dumps(response.json()['results'][0]['warnings_and_cautions'])

  return adverse_reactions, contraindications, information_for_patients, warnings_and_cautions


In [7]:
def pharm_class(unique_number,drug_ndc):
  query = drug_ndc.format(unique_number)
  response = requests.get(query)
  pharm_class = json.loads(json.dumps(response.json()['results'][0]['pharm_class']))
  
  return pharm_class

In [8]:
def recall_information(unique_number,drug_enforcement):
  query = drug_enforcement.format(unique_number)
  response = requests.get(query)
  country = json.loads(json.dumps(response.json()['results'][0]['country']))
  reason_for_recall = json.loads(json.dumps(response.json()['results'][0]['reason_for_recall']))
  return country, reason_for_recall

In [9]:
unique_number = unique_number(drug_name)
generic_name, route, substance_name, dosage_form, active_ingredients = general_information(unique_number,drug_fda)
adverse_reactions, contraindications, information_for_patients, warnings_and_cautions = specific_information(unique_number,drug_label)
pharm_cl = pharm_class(unique_number,drug_ndc)
country, reason_for_recall = recall_information(unique_number,drug_enforcement)

In [10]:
class information(dict):

  def __init__(self):
    self = dict()

  def add(self, key, value):
    self[key] = value

In [11]:
#Possible insights
insights = information()
insights.add('generic_name', generic_name)
insights.add('route', route)
insights.add('substance_name', substance_name)
insights.add('dosage_form', dosage_form)
insights.add('active_ingredients', active_ingredients)
insights.add('information_for_patients',information_for_patients)
insights.add('pharm_cl',pharm_cl)
insights.add('recall_information',recall_information)

In [12]:
insights

{'generic_name': '["AMOXICILLIN"]',
 'route': '["ORAL"]',
 'substance_name': '["AMOXICILLIN"]',
 'dosage_form': '"FOR SUSPENSION"',
 'active_ingredients': 'AMOXICILLIN',
 'information_for_patients': '["17 PATIENT COUNSELING INFORMATION Information for Patients Patients should be advised that amoxicillin may be taken every 8 hours or every 12 hours, depending on the dose prescribed. Patients should be counseled that antibacterial drugs, including amoxicillin, should only be used to treat bacterial infections. They do not treat viral infections (e.g., the common cold). When amoxicillin is prescribed to treat a bacterial infection, patients should be told that although it is common to feel better early in the course of therapy, the medication should be taken exactly as directed. Skipping doses or not completing the full course of therapy may: (1) decrease the effectiveness of the immediate treatment, and (2) increase the likelihood that bacteria will develop resistance and will not be tre

In [13]:
#Side effects and warnings
warning = information()
warning.add('adverse_reactions', adverse_reactions)
warning.add('contraindications', contraindications)
warning.add('warnings_and_cautions', warnings_and_cautions)

In [14]:
warning

{'adverse_reactions': '["6 ADVERSE REACTIONS The following are discussed in more detail in other sections of the labeling: Anaphylactic reactions [see Warnings and Precautions (5.1) ] CDAD [see Warnings and Precautions (5.2) ] The most common adverse reactions (> 1%) observed in clinical trials of amoxicillin capsules, tablets or oral suspension were diarrhea, rash, vomiting, and nausea. (6.1) To report SUSPECTED ADVERSE REACTIONS, contact Aurobindo Pharma USA, Inc. at 1-866-850-2876 or FDA at 1-800-FDA-1088 or www.fda.gov/medwatch . 6.1 Clinical Trials Experience Because clinical trials are conducted under widely varying conditions, adverse reaction rates observed in the clinical trials of a drug cannot be directly compared to rates in the clinical trials of another drug and may not reflect the rates observed in practice. The most common adverse reactions (> 1%) observed in clinical trials of amoxicillin capsules, tablets or oral suspension were diarrhea, rash, vomiting, and nausea. T

In [15]:
#alternate drug suggestion
def alternate_drugs(drug_name,drug_fda):
  url = 'https://api.fda.gov/drug/drugsfda.json?search={}&limit=10'.format(drug_name.upper())
  response = requests.get(url)
  alternate_drugs = information()
  for i in range(len(response.json()['results'])):
    temp_list = response.json()['results'][i]['products']
    for j in range(len(temp_list)):
      if(temp_list[j]['marketing_status']!='Discontinued'):
        alternate_drugs.add(temp_list[j]['brand_name'], temp_list[j]['active_ingredients'])
  return alternate_drugs

In [16]:
alternate_drugs(drug_name,drug_fda)

{'AMOXICILLIN': [{'name': 'AMOXICILLIN', 'strength': '250MG/5ML'}],
 'AMOXICILLIN AND CLAVULANATE POTASSIUM': [{'name': 'AMOXICILLIN',
   'strength': '875MG'},
  {'name': 'CLAVULANATE POTASSIUM', 'strength': 'EQ 125MG BASE'}],
 'LANSOPRAZOLE, AMOXICILLIN AND CLARITHROMYCIN (COPACKAGED)': [{'name': 'AMOXICILLIN',
   'strength': '500MG'},
  {'name': 'CLARITHROMYCIN', 'strength': '500MG'},
  {'name': 'LANSOPRAZOLE', 'strength': '30MG'}]}

In [17]:
#Contradiction
#Possible approach : compare active substances of other drugs with substances in warnings from the drug